In [1]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import random
import spacy

from tqdm import tqdm
from bs4 import BeautifulSoup

from elqm.data import get_raw_data

In [2]:
data = get_raw_data()

In [3]:
# spacy.cli.download("en_core_web_trf")

In [4]:
nlp = spacy.load("en_core_web_trf")

/home/psaegert/miniconda3/envs/elqm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Parse the HTML of each document in data and find out if there is a div with id "TexteOnly"

docs_text_only = []
docs_mixed = []
for id, doc in tqdm(data.items(), total=len(data)):
    soup = BeautifulSoup(doc['html'], 'html.parser')
    div = soup.find('div', id='TexteOnly')
    
    if div is None:
        docs_mixed.append(id)
    else:
        docs_text_only.append(id)

100%|██████████| 508/508 [00:10<00:00, 50.65it/s]


In [6]:
example_mixed = data[docs_mixed[0]]

# Remove all tags from the text
soup = BeautifulSoup(example_mixed['html'], 'html.parser')
text = soup.get_text()

# Reduce multiple line breaks to one with regex
text = re.sub(r'\n+', '\n', text)

In [7]:
doc = nlp(text)
sentences = list(doc.sents)

In [8]:
# Compute a similarity chain
# Compute the similarity between each sentence and the next one
similarity_chain = []
for i in range(len(sentences)-1):
    similarity_chain.append(sentences[i].similarity(sentences[i+1]))

/tmp/ipykernel_12064/2981476116.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_chain.append(sentences[i].similarity(sentences[i+1]))
/tmp/ipykernel_12064/2981476116.py:5: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  similarity_chain.append(sentences[i].similarity(sentences[i+1]))


In [15]:
for i, chunk in enumerate(doc.noun_chunks):
    print(i)
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

0
Official Journal Journal appos 

1
the European Union Union pobj of
2
L L appos Journal
3
COMMISSION REGULATION REGULATION appos Journal
4
EC EC appos REGULATION
5
No No appos REGULATION
6
22 July July pobj of
7
Directive Directive dobj implementing
8
EC EC appos Directive
9
the European Parliament Parliament pobj of
10
the Council Council pobj of
11
regard regard pobj with
12
ecodesign requirements requirements pobj to
13
glandless standalone circulators circulators pobj for
14
glandless circulators circulators conj circulators
15
products products pobj in
16
EEA relevance relevance pobj with
17
THE COMMISSION COMMISSION appos 

18
THE EUROPEAN COMMUNITIES COMMUNITIES pobj OF
19
regard regard dobj Having
20
the Treaty Treaty pobj to
21
the European Community Community dobj establishing
22
regard regard dobj Having
23
Directive Directive pobj to
24
EC EC appos Directive
25
the European Parliament Parliament pobj of
26
the Council Council pobj of
27
6 July July pobj of
28
a framework 

In [9]:
for i, sentence in enumerate(sentences):
    print(sentence)
    if i < len(sentences)-1:
        print("-" * 80 + f' {similarity_chain[i]:.2f}\n' )


L_2009191EN.01003501.xml
-------------------------------------------------------------------------------- 0.00


23.7.2009   
EN
-------------------------------------------------------------------------------- 0.00


Official Journal of the European Union
L 191/35
COMMISSION REGULATION (EC) No 641/2009
of 22 July 2009
implementing Directive 2005/32/EC of the European Parliament and of the Council with regard to ecodesign requirements for glandless standalone circulators and glandless circulators integrated in products
(Text with EEA relevance)
-------------------------------------------------------------------------------- 0.00


THE COMMISSION OF THE EUROPEAN COMMUNITIES,
Having regard to the Treaty establishing the European Community,
Having regard to Directive 2005/32/EC of the European Parliament and of the Council of 6 July 2005 establishing a framework for the setting of ecodesign requirements for energy-using products and amending Council Directive 92/42/EEC and Directives 96/5

In [10]:
# Get all the vectors of the sentences
vectors = []
for sentence in tqdm(sents, total=len(sents)):
    doc = nlp(sentence)
    vectors.append(doc.vector)

NameError: name 'sents' is not defined